1. Gerekli kütüphaneler

In [3]:
import joblib
import pandas as pd
from sklearn.linear_model import LogisticRegression

from flask import Flask, request, jsonify

2. Modeli kaydet

In [4]:
# Modeli örnek olarak eğitelim (varsa bu kısmı atlayabilirsin)
train_df = pd.read_csv("../datasets/normal/train_mental.csv")
X_train = train_df.drop("treatment", axis=1)
y_train = train_df["treatment"]

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Kaydet
joblib.dump(model, "mental_logreg_model.pkl")

['mental_logreg_model.pkl']